In [29]:
from langchain_huggingface import (
    HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings
)
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate


In [47]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [56]:
from pydantic import BaseModel,Field

class RewriteQueryOutputFormat(BaseModel):
  query:list=Field(...,description='The re-written user query')


rewrite_3_prompt = '''
You are an expert in rephrasing complex employee-related questions into short, compressed, and easy-to-understand alternatives.
Given a user’s original question, rewrite it into 3 distinct simplified queries that retain the core meaning but are easier to process and interpret.

Each rewritten query should:
- Be short and direct (1–2 lines)
- Avoid unnecessary words
- Use simpler language
- Capture the original intent accurately

{format_instructions}

Question: {input_question}
Answer:
'''.strip()


schema = [
    ResponseSchema(
        name='answer',
        description='List of 3 re-written queries that are short and easy to understand.'
    )
]

parser = StructuredOutputParser.from_response_schemas(schema)
format_instructions = parser.get_format_instructions()


prompt=PromptTemplate(
    template=rewrite_3_prompt,
     input_variables=['input_question'],
    partial_variables={"format_instructions": format_instructions}
)

In [57]:
llm = HuggingFaceEndpoint(
repo_id="meta-llama/Llama-3.2-3B-Instruct",
task="text-generation"
)
model = ChatHuggingFace(llm=llm)



In [58]:
chain=(
    prompt
    | model
    | parser
)


In [59]:
response=chain.invoke({
    "input_question":"pull me all the details of the person by the name unis and also the persons joining date"
})

In [60]:
print(response)

{'answer': ["What is the person's details by name Unis?", 'Who is Unis in our database?', "Can you show Unis's profile?"]}
